In [ ]:
#The scalar potential
def V0(fields, parameters, rotation):
    La = parameters['Lambdas']
    ph = parameters['phases']
    nd = np.matmul(parameters['NDW'],rotation)
    return np.sum(La * np.cos(np.matmul(nd,fields) + ph))
#Its first derivative
def V1(fields, parameters, rotation):
    La = parameters['Lambdas']
    ph = parameters['phases']
    nd = np.matmul(parameters['NDW'],rotation)
    return np.matmul(-La * np.sin(np.matmul(nd,fields) + ph),nd)
#Its second derivative
def V2(fields, parameters, rotation):
    La = parameters['Lambdas']
    ph = parameters['phases']
    nd = np.matmul(parameters['NDW'],rotation)
    return np.matmul(nd.transpose(),np.matmul(np.diag(-La * np.cos(np.matmul(nd,fields) + ph)),nd))

#Generate a random orthogonal matrix
def Random_Rotation_Matrix(n):
    r = np.random.rand(n,n)
    return np.linalg.eig(r + r.transpose())[1]

#This function defines the equation of motion
def IVP_Function(t,state):
    positions = state[:N]
    velocities = state[N:]
    position_dot = velocities
    V1_eval = V1(positions, parameters, rotation)
    velocity_dot_heavy = -V1_eval[:N_fixed] * (m2[N_fixed]/m2[:N_fixed]) -velocities[:N_fixed] * (2 * m2[N_fixed]**0.5)
    velocity_dot_light = -V1_eval[N_fixed:] - 1.5 * velocities[N_fixed:]/t
    velocity_dot = np.concatenate((velocity_dot_heavy,velocity_dot_light))
    return np.concatenate((position_dot,velocity_dot))

In [ ]:
#NUMBER OF AXIONS
N = 10

#TRIVIAL POTENTIAL
Trivial = False
#TRIVIAL INITIAL CONDITIONS:
TrivialIC = False

#FRACTIONAL DENSITY FLUCTUATION THRESHOLD FOR INTEGRATING OUT A PARTICLE
threshold = 0.01

#DOMAIN WALL NUMBERS (if not trivial)
NDW_range = 1 #Absolute value of the maximum domain wall number
p0 = 2/N #Probability that NDW = 0

#INSTANTONS
M = N**2 #Number of instantons
MUV = 1 #Scale of the UV physics
S0 = 200 #scale of the instanton actions

#DECAY CONSTANTS
f2 = np.random.rand(N) * (1 - Trivial) + Trivial #Squared decay constants
R = Random_Rotation_Matrix(N) * (1 - Trivial) + Trivial * np.eye(N)

#Domain-wall numbers if random:
if Trivial:
    M = N + 0
    NDW_matrix = np.eye(N)
    phase_list = np.zeros([1,N])[0]
else:
    NDW_matrix = np.random.choice(range(-NDW_range,NDW_range + 1), size = (M,N), p = np.concatenate((0.5 * NDW_range**-1 * (1 - p0) * np.ones(NDW_range),p0 * np.ones(1),0.5 * NDW_range**-1 * (1 - p0) * np.ones(NDW_range))))
    phase_list = 2 * np.pi * np.random.rand(M)

#Packaging these parameters into a dictionary for easy reference
parameters = {'NDW' : NDW_matrix,
             'phases' : phase_list,
             'Lambdas' : MUV**4 * np.exp(-S0 * np.random.rand(M))}

#Computing the rotation matrices
rotation_rescale = np.matmul(R.transpose(),np.diag(f2**-0.5))
rotation_rescale_inv = np.matmul(np.diag(f2**0.5),R)

#We now pick the initial condition for the fields. For now, I will assume the fundamental fields to be randomly distributed from 0 to 2 pi
theta_i = 2 * np.pi * np.random.rand(N) * (1 - TrivialIC) + 0.25 * np.pi * TrivialIC
psi = np.matmul(rotation_rescale,theta_i)

In [ ]:
#Set up the initial problem by rotating into a canonically normlized (local) mass basis
N_steps = 10000 #Number of t_i sized steps to take before checking whether to integrate out a mode
threshold = 0.01

N_fixed = 0 #This variable will count the number of fields which we have integrated out -- initialized at 0
phi_fixed = np.zeros(0)

#Calculating the rotation to the mass eigenbasis
m2,S = np.linalg.eig(V2(psi, parameters, rotation_rescale))

#Rotation matrix to the mass eigenbasis, as required for computing the gradient of the potential wrt the mass basis fields
rotation = np.matmul(rotation_rescale,S)
#Mass Eigenbasis field at its initial condition
phi = np.matmul(S.transpose(),psi)

#Use the mass eigenvalues as an estimate for the initial integration time i.e. t_0 ~ 1/m_max
t_0 = 0.01/np.max(np.abs(m2)**0.5)

#Now define the initial value problem:
initial_state = np.concatenate((phi,np.zeros(N)))

t_i,t_step = t_0 + 0,N_steps * t_0
t_f = t_step + 0
max_step = 10 * t_0


solution = sp.integrate.solve_ivp(IVP_Function, [t_i,10000], initial_state, max_step = max_step)
rho_k = np.zeros([len(solution.t),N])
rho_m = np.zeros([len(solution.t),N])

M2 = V2(solution.y[:N,-1], parameters, rotation)
m2,S = np.linalg.eig(M2)
idx = np.argsort(m2)[::-1]
m2,S = m2[idx],S[:,idx]
rotation = np.matmul(rotation,S)
i = 0
for t in solution.t:
    delta_phi = V1(np.matmul(S.transpose(),solution.y[:N,i]), parameters, rotation)
    rho_k[i] = 0.5 * np.matmul(S.transpose(),solution.y[N:,i])**2
    rho_m[i] = 0.5 * m2**-1 * delta_phi**2
    i += 1
for j in range(N):
    # plt.plot(np.log10(solution.t),np.log10(rho_m[:,j]))
    # plt.plot(np.log10(solution.t),np.log10(rho_k[:,j]))
    plt.plot(np.log10(solution.t),np.log10(rho_k[:,j] + rho_m[:,j]) + np.log10(solution.t**1.5))